<a href="https://colab.research.google.com/github/lmeloh/MetNumUN2023I/blob/main/Lab7/Week4IterativeMethodsForLinearSystemsGroup36.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple iteration for systems of linear equations

First, generate a random diagonally dominant matrix, for testing.

In [41]:
import numpy as np
rndm = np.random.RandomState(1234)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

In [42]:
A

array([[1.51915195e+01, 6.22108771e-01, 4.37727739e-01, 7.85358584e-01,
        7.79975808e-01, 2.72592605e-01, 2.76464255e-01, 8.01872178e-01,
        9.58139354e-01, 8.75932635e-01],
       [3.57817270e-01, 1.55009951e+01, 6.83462935e-01, 7.12702027e-01,
        3.70250755e-01, 5.61196186e-01, 5.03083165e-01, 1.37684496e-02,
        7.72826622e-01, 8.82641191e-01],
       [3.64885984e-01, 6.15396178e-01, 1.50753812e+01, 3.68824006e-01,
        9.33140102e-01, 6.51378143e-01, 3.97202578e-01, 7.88730143e-01,
        3.16836122e-01, 5.68098653e-01],
       [8.69127390e-01, 4.36173424e-01, 8.02147642e-01, 1.51437668e+01,
        7.04260971e-01, 7.04581308e-01, 2.18792106e-01, 9.24867629e-01,
        4.42140755e-01, 9.09315959e-01],
       [5.98092228e-02, 1.84287084e-01, 4.73552788e-02, 6.74880944e-01,
        1.55946248e+01, 5.33310163e-01, 4.33240627e-02, 5.61433080e-01,
        3.29668446e-01, 5.02966833e-01],
       [1.11894318e-01, 6.07193706e-01, 5.65944643e-01, 6.76406199e-03,
   

# I.  Jacobi iteration

Given

$$
A x = b
$$

separate the diagonal part $D$,

$$ A = D + (A - D) $$

and write

$$
x = D^{-1} (D - A) x + D^{-1} b\;.
$$

Then iterate

$$
x_{n + 1} = B x_{n} + c\;,
$$

where 

$$
B = D^{-1} (A - D) \qquad \text{and} \qquad c = D^{-1} b
$$


Let's construct the matrix and the r.h.s. for the Jacobi iteration

In [43]:
diag_1d = np.diag(A)

B = -A.copy()
np.fill_diagonal(B, 0)

D = np.diag(diag_1d)
invD = np.diag(1./diag_1d)
BB = invD @ B 
c = invD @ b

In [44]:
# sanity checks
from numpy.testing import assert_allclose

assert_allclose(-B + D, A)


# xx is a "ground truth" solution, compute it using a direct method
xx = np.linalg.solve(A, b)

np.testing.assert_allclose(A@xx, b)
np.testing.assert_allclose(D@xx, B@xx + b)
np.testing.assert_allclose(xx, BB@xx + c)

Check that $\| B\| \leqslant 1$:

In [45]:
np.linalg.norm(BB)

0.36436161983015336

### Do the Jacobi iteration

In [46]:
n_iter = 50

x0 = np.ones(n)
x = x0
for _ in range(n_iter):
    x = BB @ x + c

In [47]:
# Check the result:

A @ x - b

array([ 1.11022302e-16,  0.00000000e+00, -2.22044605e-16, -1.11022302e-16,
        1.11022302e-16,  0.00000000e+00, -2.42861287e-17,  0.00000000e+00,
       -2.77555756e-17,  1.11022302e-16])

### Task I.1

Collect the proof-of-concept above into a single function implementing the Jacobi iteration. This function should receive the r.h.s. matrix $A$, the l.h.s. vector `b`, and the number of iterations to perform.


The matrix $A$ in the illustration above is strongly diagonally dominant, by construction. 
What happens if the diagonal matrix elements of $A$ are made smaller? Check the convergence of the Jacobi iteration, and check the value of the norm of $B$.

(20% of the total grade)


In [48]:
# ... ENTER YOUR CODE HERE ...
def jacobi(A, b, num_iter=1000):
    n = A.shape[0]
    diag_A = np.diag(A)
    B = np.zeros_like(A)
    
    for i in range(n):
        for j in range(n):
            if i != j:
                B[i, j] = -A[i, j] / diag_A[i]
    
    c = b / diag_A
    norm = np.linalg.norm(B)
    
    x0 = np.ones(n)
    x = x0
    for _ in range(num_iter):
        x = B @ x + c
    
    return x, norm


jacobi(A, b)


(array([ 0.03919429,  0.03780037,  0.04283232,  0.02365951,  0.05745031,
        -0.00030244, -0.00577279,  0.03177549, -0.00422849,  0.05284648]),
 0.36436161983015336)

# II. Seidel's iteration.

##### Task II.1

Implement the Seidel's iteration. 

Test it on a random matrix. Study the convergence of iterations, relate to the norm of the iteration matrix.

(30% of the total grade)

In [49]:
import matplotlib.pyplot as plt

def gauss_seidel(A, b, X_i, eps, max_iter):

  x = X_i.copy()

  for i in range(max_iter):

    for j in range(A.shape[0]):
      x[j] = (b[j] - np.dot(A[j, :j], x[:j])) / A[j, j]

    norm_x = np.linalg.norm(x - X_i)
    if norm_x < eps:
      break

  return x

A = np.random.rand(10, 10)
b = np.random.rand(10)
print(A)
x = gauss_seidel(A, b, np.zeros(10), 1e-6, 100)

true_x = np.linalg.solve(A, b)

error = np.linalg.norm(x - true_x)
print(error)

error_list = []
for i in range(1, 100):
  x = gauss_seidel(A, b, np.zeros(10), 1e-6, i)
  error_list.append(np.linalg.norm(x - true_x))


[[0.62769607 0.47833613 0.15052327 0.44662373 0.43474672 0.90355587
  0.37956481 0.81507912 0.13108238 0.24890979]
 [0.2933705  0.56655688 0.82770487 0.32141662 0.26204165 0.59759696
  0.48736437 0.7054866  0.99644329 0.95495365]
 [0.69558304 0.92706007 0.10532988 0.44237098 0.0852581  0.45358325
  0.79674373 0.01727729 0.13010803 0.07905847]
 [0.64730885 0.74790675 0.98582573 0.78440304 0.98570155 0.84598173
  0.17711475 0.14372273 0.19023984 0.41290578]
 [0.89972379 0.23524983 0.78117543 0.806877   0.40495023 0.30895986
  0.63104521 0.11532339 0.3009346  0.83154567]
 [0.51638505 0.14521472 0.35682768 0.80764041 0.19008308 0.25194925
  0.88125699 0.49423025 0.29600593 0.06152939]
 [0.16179792 0.0152095  0.49652963 0.08602267 0.32199873 0.79014381
  0.21604676 0.44878989 0.77516928 0.52487268]
 [0.87558431 0.95388584 0.51796844 0.40192003 0.51048022 0.12305721
  0.75776172 0.16582121 0.0573824  0.79763935]
 [0.27010652 0.03209271 0.92163859 0.53448206 0.01682822 0.86764383
  0.30191929

# III. Minimum residual scheme

### Task III.1

Implement the $\textit{minimum residual}$ scheme: an explicit non-stationary method, where at each step you select the iteration parameter $\tau_n$ to minimize the residual $\mathbf{r}_{n+1}$ given $\mathbf{r}_n$. Test it on a random matrix, study the convergence to the solution, in terms of the norm of the residual and the deviation from the ground truth solution (which you can obtain using a direct method). Study how the iteration parameter $\tau_n$ changes as iterations progress.

(50% of the grade)

In [50]:
import numpy as np

def minres(A, b, n_iter=500, eps=1e-07):
    x = np.zeros_like(b)
    r = b - A @ x
    p = r.copy()
    T = []
    
    for i in range(n_iter):
        Ap = A @ p
        alpha = np.dot(r, r) / np.dot(p, Ap)
        x = x + alpha * p
        r = r - alpha * Ap
        beta = np.dot(r, r) / np.dot(p, Ap)
        p = r + beta * p
        T.append(alpha)
        
        if np.linalg.norm(r) < eps:
            break
    
    return x, T


x, T = minres(A, b)

print("Solución:")
print(x)
print("Paramentos de iteracion: ")
print(T)



#Pruebas para el vector solucion
rtol = 1e-5
atol = 1e-5


xx = np.linalg.solve(A, b)

np.testing.assert_allclose(A@x, b, rtol=rtol, atol=atol)
np.testing.assert_allclose(x, xx, rtol=rtol, atol=atol)

Solución:
[-4.44822438e+67 -7.27627994e+67 -1.14617259e+68 -3.51617941e+67
 -9.99307487e+67 -1.00919915e+68 -9.94818107e+66 -1.14072248e+68
 -2.52497723e+67 -6.88116043e+67]
Paramentos de iteracion: 
[0.24389601207037648, 4.0415054029090625, 0.01874689278658867, 0.9963402742733356, -0.43533746419048636, -1.1528962174617137, -0.04360830945492343, 0.21057521987413147, -2.4907451063370516, -0.03690811942878977, 0.9475562040730419, 0.7519101739222082, -0.30849434053152647, 0.2446878703349482, 1.9673912776639682, 0.40735604169072825, 2.1995134657433035, 0.013536649656789614, 1.8704239418153377, 0.10884213701845222, -0.07239393651030206, 0.22400533639587675, -1.4737753665903457, -0.13183041681164862, 0.27776757268998287, 2.6295228045794348, 0.0547669378950454, -0.2932203753083185, 0.29355523752133955, 2.797035638800494, 0.019479974659858987, 0.765118376633564, -0.5228696666161369, 0.3425310264894693, 1.1841533558748794, -0.17325594281028245, 0.33379153394813144, -3.7112960058204503, -0.00253

AssertionError: ignored